In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
import re
from datetime import datetime

# Function to parse travelers string (e.g., "2 adults, 1 child" -> {'adults': 2, 'children': 1})
def parse_travelers(travelers_str):
    travelers_dict = {'adults': 1, 'children': 0, 'infants': 0}  # Defaults
    # Simple regex to extract numbers and types
    matches = re.findall(r'(\d+)\s*(adult|adults|child|children|infant|infants)', travelers_str.lower())
    for num, typ in matches:
        num = int(num)
        if typ in ['adult', 'adults']:
            travelers_dict['adults'] = num
        elif typ in ['child', 'children']:
            travelers_dict['children'] = num
        elif typ in ['infant', 'infants']:
            travelers_dict['infants'] = num
    return travelers_dict

# Helper function to type slowly (simulates human input)
def type_slowly(element, text, delay=0.1):
    element.click()
    element.clear()
    for char in text:
        element.send_keys(char)
        time.sleep(delay)
    element.send_keys(Keys.ENTER)
    time.sleep(1)

# Prompt user for inputs
print("Welcome to the Kayak Flight Search Automator!")
origin = input("Enter origin (e.g., New York): ").strip()
destination = input("Enter destination (e.g., London): ").strip()
departure_date_input = input("Enter departure date (YYYY-MM-DD, e.g., 2025-12-01): ").strip()
return_date_input = input("Enter return date (YYYY-MM-DD, e.g., 2025-12-10): ").strip()
travelers = input("Enter travelers (e.g., 1 adult or 2 adults, 1 child): ").strip()

# Validate and format inputs
if not origin or not destination:
    print("Origin and destination are required.")
    exit(1)

try:
    departure_date = datetime.strptime(departure_date_input, '%Y-%m-%d').strftime('%Y-%m-%d')
except ValueError:
    print("Departure date must be in YYYY-MM-DD format.")
    exit(1)

try:
    return_date = datetime.strptime(return_date_input, '%Y-%m-%d').strftime('%Y-%m-%d')
except ValueError:
    print("Return date must be in YYYY-MM-DD format.")
    exit(1)

print("\nStarting automation...")

# Set up Chrome options (optional: add headless if you want it to run in background)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Uncomment to run without opening browser window
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get("https://www.kayak.com/flights")

wait = WebDriverWait(driver, 20)  # Increased wait time for reliability

try:
    # Fill 'From' field
    from_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="From?"]')))
    type_slowly(from_input, origin)

    # Fill 'To' field
    to_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="To?"]')))
    type_slowly(to_input, destination)

    # Fill Departure Date
    depart_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="Depart"]')))
    type_slowly(depart_input, departure_date)

    # Fill Return Date
    return_input = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[placeholder="Return"]')))
    type_slowly(return_input, return_date)

    # Handle Travelers
    traveler_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div[aria-label="Travelers"]')))
    traveler_button.click()
    time.sleep(2)  # Wait for dropdown to open

    travelers_parsed = parse_travelers(travelers)

    # Assuming Kayak has + and - buttons for adults, children, etc.
    # Adjust selectors based on actual page structure (inspect element to confirm)
    # For adults
    adult_plus = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Adults')]/following-sibling::div//button[@aria-label='Increase']")))
    adult_minus = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Adults')]/following-sibling::div//button[@aria-label='Decrease']")))

    # Reset to 1 adult first (click minus until 1, but assuming starts at 1)
    # This is simplistic; in reality, you might need to read current value
    for _ in range(9):  # Max 9, so click minus a lot to reset
        try:
            adult_minus.click()
            time.sleep(0.1)
        except:
            break
    # Now add to desired number
    for _ in range(travelers_parsed['adults'] - 1):
        adult_plus.click()
        time.sleep(0.1)

    # Similarly for children (if needed)
    if travelers_parsed['children'] > 0:
        child_plus = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Children')]/following-sibling::div//button[@aria-label='Increase']")))
        for _ in range(travelers_parsed['children']):
            child_plus.click()
            time.sleep(0.1)

    # Similarly for infants (if needed)
    if travelers_parsed['infants'] > 0:
        infant_plus = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(text(), 'Infants')]/following-sibling::div//button[@aria-label='Increase']")))
        for _ in range(travelers_parsed['infants']):
            infant_plus.click()
            time.sleep(0.1)

    # Close the dropdown (assuming clicking outside or a done button)
    # Sometimes pressing ESC or clicking elsewhere works
    traveler_button.send_keys(Keys.ESCAPE)
    time.sleep(1)

    # Click Search
    search_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label*="Search flights"]')))
    search_button.click()

    # Wait for results to load (adjust time as needed)
    time.sleep(15)  # Or use wait for a specific element on results page

    print("Flight search completed. Results should be loaded in the browser.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    driver.quit()

Welcome to the Kayak Flight Search Automator!


Enter origin (e.g., New York):  Londen
Enter destination (e.g., London):  Toronto
Enter departure date (YYYY-MM-DD, e.g., 2025-12-01):  2025-12-1
Enter return date (YYYY-MM-DD, e.g., 2025-12-10):  2025-12-30
Enter travelers (e.g., 1 adult or 2 adults, 1 child):  1 adult



Starting automation...
An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.204)
Stacktrace:
#0 0x6467484a11fa <unknown>
#1 0x646747fb1810 <unknown>
#2 0x646747f8748f <unknown>
#3 0x64674802c9bd <unknown>
#4 0x6467480429bc <unknown>
#5 0x646748024323 <unknown>
#6 0x646747ff2de0 <unknown>
#7 0x646747ff3dbe <unknown>
#8 0x64674846d12b <unknown>
#9 0x6467484710c7 <unknown>
#10 0x64674845a6cc <unknown>
#11 0x646748471c47 <unknown>
#12 0x64674843f67f <unknown>
#13 0x646748490288 <unknown>
#14 0x646748490450 <unknown>
#15 0x6467484a0076 <unknown>
#16 0x7af3d9094ac3 <unknown>



In [25]:
import re
import sys

print("===== WEB SECURITY SCANNER (DEFENSIVE ONLY) =====")
print("Paste your web logs or URLs below.")
print("Type 'DONE' when finished:\n")

logs = ""
while True:
    line = input()
    if line.strip().upper() == "DONE":
        break
    logs += line + "\n"


# -----------------------------
# ATTACK SIGNATURES (SAFE ONLY)
# -----------------------------

ATTACKS = {
    "SQL Injection": [
        r"UNION\s+SELECT",
        r"OR\s+1=1",
        r"'--",
        r"sleep\(\d+\)",
        r"SELECT.+FROM"
    ],

    "Cross-Site Scripting (XSS)": [
        r"<script>",
        r"alert\(",
        r"onerror=",
        r"%3Cscript%3E"
    ],

    "Remote Code Execution (RCE)": [
        r"cmd=",
        r"exec=",
        r"system\(",
        r";cat",
        r";ls",
        r"&&id",
        r"wget\s+http"
    ],

    "Directory Traversal": [
        r"\.\./\.\./",
        r"%2e%2e%2f",
        r"etc/passwd",
        r"boot.ini"
    ],

    "Command Injection": [
        r";whoami",
        r";uname",
        r";cat",
        r"\|\s*nc",
        r"\|\s*bash"
    ],

    "File Inclusion": [
        r"php://",
        r"data:text",
        r"file=",
        r"include="
    ]
}

# -----------------------------
# DETECTION ENGINE
# -----------------------------

detected = {}

for attack, rules in ATTACKS.items():
    for rule in rules:
        if re.search(rule, logs, re.IGNORECASE):
            detected.setdefault(attack, 0)
            detected[attack] += 1

# -----------------------------
# REPORTING
# -----------------------------

print("\n===============================")
print("      SECURITY SCAN REPORT")
print("===============================\n")

if not detected:
    print("No threats found. Logs appear normal.")
else:
    print("⚠️  Detected Threats:")
    for attack, count in detected.items():
        print(f"- {attack}: {count} matches")

risk_level = (
    "LOW" if not detected
    else "MEDIUM" if len(detected) <= 2
    else "HIGH"
)

print(f"\nRISK LEVEL: {risk_level}")

print("\n=== RECOMMENDATIONS ===")

if not detected:
    print("- Continue monitoring. No issues detected.")
else:
    if "SQL Injection" in detected:
        print("- Enable prepared statements / parameterized queries.")
    if "XSS" in detected:
        print("- Apply output escaping & CSP headers.")
    if "Directory Traversal" in detected:
        print("- Block '..' patterns and sanitize paths.")
    if "RCE" in detected:
        print("- Validate input and block shell metacharacters (; | &&).")
    if "Command Injection" in detected:
        print("- Disable shell execution in your application.")
    if "File Inclusion" in detected:
        print("- Disable remote file includes in PHP settings.")

print("\n=====================================")


===== WEB SECURITY SCANNER (DEFENSIVE ONLY) =====
Paste your web logs or URLs below.
Type 'DONE' when finished:



 2025-11-25 15:33:42 GET /download?file=../../../../etc/passwd src=44.51.21.18 2025-11-25 15:33:43 GET /view?file=%2e%2e%2f%2e%2e%2fconfig.php src=44.51.21.18 2025-11-25 15:33:44 GET /img?path=../windows/win.ini src=92.10.55.4
 DONE



      SECURITY SCAN REPORT

⚠️  Detected Threats:
- Directory Traversal: 3 matches
- File Inclusion: 1 matches

RISK LEVEL: MEDIUM

=== RECOMMENDATIONS ===
- Block '..' patterns and sanitize paths.
- Disable remote file includes in PHP settings.

